In [3]:
!pip install -qU  langchain-core
!pip install -qU langchain-community
!pip install -qU  crewai[tools] 
!pip install -qU langchain-groq

In [4]:
pip -qU install python-dotenv

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\Anoop Maurya\Documents\GitHub\SQL-Agent-using-CrewAI-and-Groq\.venv\Scripts\python.exe -m pip <command> [options]

no such option: -U


In [3]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,MI,FT,Data Analyst,1125000,INR,14329,IN,100,IN,L
1,2022,EN,FT,Data Scientist,130000,USD,130000,US,0,US,M
2,2023,MI,FT,Data Analyst,150000,USD,150000,US,0,US,M
3,2023,MI,FT,Data Analyst,100000,USD,100000,US,0,US,M
4,2023,SE,FT,Machine Learning Engineer,261500,USD,261500,US,0,US,L


In [5]:
connection = sqlite3.connect("salaries.db")
df.to_sql(name="salaries", con=connection)

1948

In [6]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str


def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()


class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")


In [7]:
llm = ChatGroq(
    temperature=0,
    #model_name="llama3-70b-8192",
    model_name="mixtral-8x7b-32768",
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)

In [8]:
db = SQLDatabase.from_uri("sqlite:///salaries.db")

In [9]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

Using Tool: list_tables


'salaries'

In [10]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print(tables_schema.run("salaries"))

Using Tool: tables_schema

CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2022	MI	FT	Data Analyst	1125000	INR	14329	IN	100	IN	L
1	2022	EN	FT	Data Scientist	130000	USD	130000	US	0	US	M
2	2023	MI	FT	Data Analyst	150000	USD	150000	US	0	US	M
*/


In [11]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

Using Tool: execute_sql


"[(0, 2022, 'MI', 'FT', 'Data Analyst', 1125000, 'INR', 14329, 'IN', 100, 'IN', 'L'), (1, 2022, 'EN', 'FT', 'Data Scientist', 130000, 'USD', 130000, 'US', 0, 'US', 'M'), (2, 2023, 'MI', 'FT', 'Data Analyst', 150000, 'USD', 150000, 'US', 0, 'US', 'M'), (3, 2023, 'MI', 'FT', 'Data Analyst', 100000, 'USD', 100000, 'US', 0, 'US', 'M'), (4, 2023, 'SE', 'FT', 'Machine Learning Engineer', 261500, 'USD', 261500, 'US', 0, 'US', 'L')]"

In [12]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'The SQL query provided does not contain any of the common mistakes mentioned. However, it is missing the table name in the `SELECT` clause and the `FROM` keyword. Here is the corrected query:\n\n```sql\nSELECT * \nFROM salaries \nWHERE salary > 10000 \nLIMIT 5;\n```'

In [13]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [14]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

In [15]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

In [16]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [17]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [18]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [19]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="crew.log",
)

In [20]:
inputs = {
    "query": "Effects on salary (in USD) based on company location, size and employee experience"
}

result = crew.kickoff(inputs=inputs)

 [2024-07-15 07:36:01][DEBUG]: == Working Agent: Senior Database Developer
 [2024-07-15 07:36:01][INFO]: == Starting Task: Extract data that is required for the query Effects on salary (in USD) based on company location, size and employee experience.
 

salaries

 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2022	MI	FT	Data Analyst	1125000	INR	14329	IN	100	IN	L
1	2022	EN	FT	Data Scientist	130000	USD	130000	US	0	US	M
2	2023	MI	FT	Data Analyst	150000	USD	150000	US	0	US	M
*/



<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'


 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Check_Sql
sql_query
  field required (type=value_error.missing).
 Tool check_sql accepts these inputs: check_sql(sql_query: 'string') -      Use this tool to double check if your query is correct before executing it. Always use this     tool before executing a query with `execute_sql`.     



<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'


 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Execute_Sql
sql_query
  field required (type=value_error.missing).
 Tool execute_sql accepts these inputs: execute_sql(sql_query: 'string') - Execute a SQL query against the database. Returns the result



<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'
<unknown>:1: SyntaxWarning: invalid escape sequence '\_'


 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Execute_Sql
sql_query
  field required (type=value_error.missing).
 Tool execute_sql accepts these inputs: execute_sql(sql_query: 'string') - Execute a SQL query against the database. Returns the result

 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Check_Sql
sql_query
  field required (type=value_error.missing).
 Tool check_sql accepts these inputs: check_sql(sql_query: 'string') -      Use this tool to double check if your query is correct before executing it. Always use this     tool before executing a query with `execute_sql`.     



In [ ]:
result